## Rozwiązywanie układów równań liniowych metodami interacyjnymi 

### Zadanie 1
Zaimplementuj metodę Jacobiego. Podaj warunki jej stosowalności. Wygeneruj co najmniej trzy odpowiednie macierze o różnych wielkościach i sprawdź działanie swojej metody. Zwróć uwagę na zbieżność tej metody. 





In [0]:
import numpy as np

def jacobi(A, b, max_iterations):
    D = np.diag(np.diag(A))
    LU = A - D
    x = np.zeros(len(b))
    epsilon = 1e-10
    for i in range(max_iterations):
        D_inv = np.diag(1 / np.diag(D))
        x_new = np.dot(D_inv, b - np.dot(LU, x))
        if np.linalg.norm(x_new - x) < epsilon:
            return x_new
        x = x_new
    return x

Metodę Jakobiego rozwiązywania równań liniowych można zastosować, gdy macierz A jest macierzą diagonalnie zdominowaną. (Macierz A jest macierzą diagonalnie zdominowaną, gdy dla każdego jej wiersza wartość leżąca na jego przecięciu z diagonalą jest większa od sumy wszystkich pozostałych wartości w tym wierszu).

Metoda Jacobiego jest zbieżna dla każdej macierzy nieredukowalnej i diagonalnie zdominowanej A. (Macierz A jest macierzą nieredukowalną, jeżeli nie istnieje możliwość sprowadzenia jej do postaci trójkątnej górnej zamieniając jej kolumny i wiersze).

In [0]:
import random

def random_matrix(min, max, n):
    A = np.zeros((n, n))
    for i in range(n):
        sum = 0
        for j in range(n):
            if j >= i: 
              A[i][j] = A[j][i] = random.randint(min, max)
            if i != j: 
              sum += abs(A[i][j])
        A[i][i] = sum
    return A


def random_vector(min, max, n):
    return np.random.uniform(low=min, high=max, size=(n,))

In [21]:
def compare(A, b, max_iterations):
  print("numpy result:")
  print(np.linalg.solve(A,b))
  print("Jacobi method result:")
  print(jacobi(A, b, max_iterations), '\n')

A = random_matrix(-10, 10, 4)
b = random_vector(-10, 10, 4)
compare(A, b, 1410)

A = random_matrix(-40, 40, 8)
b = random_vector(-40, 40, 8)
compare(A, b, 1410)

A = random_matrix(-200, 200, 16)
b = random_vector(-200, 200, 16)
compare(A, b, 1410)

numpy result:
[-0.57930491 -0.57263982 -1.47049441  0.85734616]
Jacobi method result:
[-0.57930491 -0.57263982 -1.47049441  0.85734616] 

numpy result:
[ 0.20783882  0.26231904  0.26953528 -0.17858558 -0.09802385 -0.18196634
  0.06731975  0.07314173]
Jacobi method result:
[ 0.20783882  0.26231904  0.26953528 -0.17858558 -0.09802385 -0.18196634
  0.06731975  0.07314173] 

numpy result:
[ 0.02685936 -0.0181756   0.12170074  0.05407556 -0.04199882 -0.08369706
  0.04283578  0.13974698  0.13092198 -0.11569997  0.20134443 -0.08745933
 -0.0751705  -0.16235314  0.18878432 -0.0030292 ]
Jacobi method result:
[ 0.02685936 -0.0181756   0.12170074  0.05407556 -0.04199882 -0.08369706
  0.04283578  0.13974698  0.13092198 -0.11569997  0.20134443 -0.08745933
 -0.0751705  -0.16235314  0.18878432 -0.0030292 ] 



### Zadanie 2
Zaimplementuj metodę Gaussa-Seidla i kolejnych nadrelaksacji (successive over-relaxation). Podaj warunki stosowalności tych metod. Przeprowadź badanie działania swoich implementacji analogicznie jak w poprzednim zadaniu. Porównaj zbieżność wszystkich trzech metod. 

In [0]:
def gauss_seidl(A, b, max_iterations):
    x = np.zeros(len(b))
    for i in range(max_iterations):
        x_new = np.zeros_like(x)
        for i in range(A.shape[0]):
            s1 = np.dot(A[i, :i], x_new[:i])
            s2 = np.dot(A[i, i + 1:], x[i + 1:])
            x_new[i] = (b[i] - s1 - s2) / A[i, i]
        if np.allclose(x, x_new, rtol=1e-8):
            break
        x = x_new
    return x


def sor(A, b, max_iterations, omega):
    epsilon = 1e-8
    x = np.zeros_like(b)
    if omega < 0 or omega > 2:
        print('omega < 0 or omega > 2')
        return [x, -1]
    n = b.shape
    x_new = np.zeros_like(x)
    for _ in range(max_iterations):
        for i in range(n[0]):
            new_values_sum = np.dot(A[i, :i], x[:i])
            old_values_sum = np.dot(A[i, i + 1:], x_new[i + 1:])
            x[i] = (b[i] - (old_values_sum + new_values_sum)) / A[i, i]
            x[i] = np.dot(x[i], omega) + np.dot(x_new[i], (1 - omega))
        if np.linalg.norm(np.dot(A, x) - b) < epsilon:
            break
        x_new = x
    return x

Metoda Jacobiego - zbieżna gdy macierz A jest:
- nieredukowalna
- diagonalnie zdominowana

Metoda Gaussa-Seidela - zbieżna w tych samych przypadkach, w których zbieżna jest metoda Jacobiego a ponadto, jeżeli macierz A jest dodatnio określona, to metoda Gaussa-Seidela jest zbieżna dla dowolengo wektora początkowego.

Metoda kolejnych nadrelaksacji (successive over-relaxation) - jest zmodyfikowaną metodą Gaussa-Seidela, która wykorzystując parametr $\omega$ należący do przedziału (0, 2) może przyspieszyć jej zbieżność. (dla wartośći $\omega$ spoza zakresu (0, 2) metoda może być rozbieżna.

In [22]:
def compare(A, b, max_iterations, omega):
  print("numpy result:")
  print(np.linalg.solve(A,b))
  print("Gauss-Seidl method result:")
  print(gauss_seidl(A, b, max_iterations))
  print("SOR method result:")
  print(sor(A, b, max_iterations, omega), '\n')

A = random_matrix(-10, 10, 4)
b = random_vector(-10, 10, 4)
compare(A, b, 1410, 0.69)

A = random_matrix(-40, 40, 8)
b = random_vector(-40, 40, 8)
compare(A, b, 1410, 0.69)

A = random_matrix(-200, 200, 16)
b = random_vector(-200, 200, 16)
compare(A, b, 1410, 0.69)

numpy result:
[ 0.58370841  0.02681287 -0.76730795  0.88305142]
Gauss-Seidl method result:
[ 0.58370838  0.02681289 -0.76730792  0.88305139]
SOR method result:
[ 0.58370841  0.02681287 -0.76730795  0.88305142] 

numpy result:
[ 0.33640958  0.09183796  0.21230273  0.53018889  0.47343679  0.51051271
 -0.07863289  0.06440692]
Gauss-Seidl method result:
[ 0.33640957  0.09183797  0.21230272  0.53018888  0.47343678  0.51051271
 -0.0786329   0.06440692]
SOR method result:
[ 0.33640958  0.09183796  0.21230273  0.53018889  0.47343679  0.51051271
 -0.07863289  0.06440692] 

numpy result:
[-0.07413723 -0.05087741  0.04301569 -0.09597676 -0.06622471  0.10754698
  0.09214374 -0.01557891 -0.09136871  0.05739217  0.12065272 -0.01562072
  0.05990252 -0.10247509 -0.06186244  0.09701591]
Gauss-Seidl method result:
[-0.07413723 -0.05087742  0.04301569 -0.09597676 -0.06622471  0.10754697
  0.09214374 -0.01557891 -0.09136871  0.05739217  0.12065272 -0.01562072
  0.05990252 -0.10247509 -0.06186244  0.097015

Zbiezność metod badam na podstawie ilości wykonywanych przez nie iteracji.

In [0]:
def jacobi_iter(A, b, max_iterations):
    iterations = 0
    D = np.diag(np.diag(A))
    LU = A - D
    x = np.zeros(len(b))
    epsilon = 1e-10
    for i in range(max_iterations):
        iterations += 1
        D_inv = np.diag(1 / np.diag(D))
        x_new = np.dot(D_inv, b - np.dot(LU, x))
        if np.linalg.norm(x_new - x) < epsilon:
            return iterations
        x = x_new
    return iterations


def gauss_seidl_iter(A, b, max_iterations):
    iterations = 0
    x = np.zeros(len(b))
    for i in range(max_iterations):
        iterations += 1
        x_new = np.zeros_like(x)
        for i in range(A.shape[0]):
            s1 = np.dot(A[i, :i], x_new[:i])
            s2 = np.dot(A[i, i + 1:], x[i + 1:])
            x_new[i] = (b[i] - s1 - s2) / A[i, i]
        if np.allclose(x, x_new, rtol=1e-8):
            break
        x = x_new
    return iterations


def sor_iter(A, b, max_iterations, omega):
    iterations = 0
    epsilon = 1e-8
    x = np.zeros_like(b)
    if omega < 0 or omega > 2:
        print('omega < 0 or omega > 2')
        return [x, -1]
    n = b.shape
    x_new = np.zeros_like(x)
    for _ in range(max_iterations):
        iterations += 1
        for i in range(n[0]):
            new_values_sum = np.dot(A[i, :i], x[:i])
            old_values_sum = np.dot(A[i, i + 1:], x_new[i + 1:])
            x[i] = (b[i] - (old_values_sum + new_values_sum)) / A[i, i]
            x[i] = np.dot(x[i], omega) + np.dot(x_new[i], (1 - omega))
        if np.linalg.norm(np.dot(A, x) - b) < epsilon:
            break
        x_new = x
    return iterations

In [48]:
def compare_iterations(A, b, max_iterations, omega):
  print(f"Jacobi method iterations: {jacobi_iter(A, b, max_iterations)}")
  print(f"Gaus-Seidl method iterations: {gauss_seidl_iter(A, b, max_iterations)}")
  print(f"SOR method iterations: {sor_iter(A, b, max_iterations, omega)}, \n")

A = random_matrix(-10, 10, 24)
b = random_vector(-10, 10, 24)
compare_iterations(A, b, 1410, 0.69)

A = random_matrix(-20, 240, 124)
b = random_vector(-20, 240, 124)
compare_iterations(A, b, 1410, 0.69)

A = random_matrix(-20, 400, 256)
b = random_vector(-20, 400, 256)
compare_iterations(A, b, 1410, 0.69)

Jacobi method iterations: 26
Gaus-Seidl method iterations: 11
SOR method iterations: 13, 

Jacobi method iterations: 1403
Gaus-Seidl method iterations: 10
SOR method iterations: 17, 

Jacobi method iterations: 1410
Gaus-Seidl method iterations: 10
SOR method iterations: 18, 

